# 依存关系和依存文法

**短语结构文法**是关于词和词序列如何结合起来形成句子成分的。一个独特和互补的方式，**依存文法**，集中关注的是词与其他词之间的关系。**依存关系**是一个中心词和它的依赖之间的二元对称关系。一个句子的**中心词**通常是动词，所有其他词要么依赖于中心词，要么通过一系列的依赖路径与中心词连通。

依存关系可以用加标签的有向图来表示：其中节点是词汇项；加标签的箭头表示依赖关系，从中心词到依赖，箭头的标签表示依赖与它的中心词之间的语法功能，例如 I 是 shot（整个句子的中心词）的 SBJ（主语），in 是 elephant 的 NMOD（名词修饰词）。

![depgraph0.png](resources/depgraph0.png)

与短语结构文法类似，依存文法可以通过 [nltk.DependencyGrammar](https://www.nltk.org/_modules/nltk/grammar.html#DependencyGrammar) 来构造：

In [1]:
import nltk

groucho_dep_grammar = nltk.DependencyGrammar.fromstring("""
    'shot' -> 'I' | 'elephant' | 'in'
    'elephant' -> 'an' | 'in'
    'in' -> 'pajamas'
    'pajamas' -> 'my'
""")

print(groucho_dep_grammar)

Dependency grammar with 7 productions
  'shot' -> 'I'
  'shot' -> 'elephant'
  'shot' -> 'in'
  'elephant' -> 'an'
  'elephant' -> 'in'
  'in' -> 'pajamas'
  'pajamas' -> 'my'


依存关系图是一个**投影**，当所有词都按线性顺序书写，我们可以在词上绘制箭头而不会交叉。这等于说一个词及其所有后代依赖（依赖及其依赖的依赖等）在句子中形成一个连续的词序列。我们可以使用投影依存关系分析器 [nltk.ProjectiveDependencyParser](http://www.nltk.org/_modules/nltk/parse/projectivedependencyparser.html#ProjectiveDependencyParser) 来分析句子：

In [2]:
pdp = nltk.ProjectiveDependencyParser(groucho_dep_grammar)
sent = 'I shot an elephant in my pajamas'.split()
trees = pdp.parse(sent)
for tree in trees:
    print(tree)

(shot I (elephant an (in (pajamas my))))
(shot I (elephant an) (in (pajamas my)))


这些括号括起来的依存关系结构可以用树结构表示，依赖作为中心词的孩子节点，如下图所示：

![ch08-tree-10.png](resources/ch08-tree-10.png)

关于在一个成分 C 中如何确定哪个是中心词 H，哪个是依赖 D，已经提出了很多不同的标准，最重要的有以下几种：
1. H 决定类型 C 的分布；或者 C 的外部句法属性取决于 H。
2. H 定义 C 的语义类型。
3. H 必须有而 D 是可选的。
4. H 选择 D 并决定它是必须有的还是可选的。
5. D 的形态由 H 决定。

当我们在一个短语结构文法中说一个 PP 的直接成分是 P 和 NP 时，我们隐含了中心词/依赖之间的区别。介词短语是一个短语，它的中心词是一个介词，NP 是 P 的依赖。同样的区别在我们已经讨论过的其它类型的短语结构文法中也存在。

## 配价与词汇

我们观察下面 4 个例子中动词和它们的依赖：

a. The squirrel was frightened.

b. Chatterer saw the bear.

c. Chatterer thought Buster was angry.

d. Joe put the fish on the log.

| 产生式        | 中心词汇 |
|---------------|----------|
| VP -> V Adj   | was      |
| VP -> V NP    | saw      |
| VP -> V S     | thought  |
| VP -> V NP PP | put      |

可以看出 was 可以与跟在其后的形容词一起出现，saw 可以与跟在其后的 NP 一起出现，thought 可以与跟在其后的 S 一起出现，put 可以与跟在其后的 NP 和 PP 一起出现。依赖类型 Adj、NP、PP 和 S 通常被称为各动词的**补语**，什么动词可以和什么补语一起出现具有很强的约束，因此下面的词序列是不符合语法规则的：

a. The squirrel was Buster was angry.

b. Chatterer saw frightened.

c. Chatterer thought the bear.

d. Joe put on the log.

在依存文法得传统中，上述动词被认为具有不同得**配价**，配价限制不仅适用于动词，也适用于其他类得中心词。

在短语结构文法的框架下，我们也可以通过一些方法来排除这类不合语法的例子。我们可以将动词划分为更多“子类别”，每个子类别与一组不同的补语关联。例如：**及物动词**，如 chased 和 saw，需要后面跟 NP 对象补语。我们为及物动词引入一个新的类别标签，叫做 TV（transitive verb）：

    VP -> TV NP
    TV -> 'chased' | 'saw'
    
通过这样的产生式，我们就可以排除例如 Chatterer thought the bear 这样的错误语法了。

## 扩大规模

到目前为止我们只考虑了“玩具文法”，如果要想将文法扩大到覆盖自然语言的大型语料库将会是一件非常困难的事情。即使我们允许使用各种可以提供更简洁的文法产生式的形式化工具，保持对覆盖一种语言的主要成分所需要的众多产生式之间复杂相互作用的控制仍然是极其困难的。换句话说，很难将文法模块化，每部分文法独立开发。另一个困难是当文法扩展到包括更加广泛的成分时，适用于任何一个句子的分析数量也相应的增加，也就是说歧义随着覆盖而增加。